In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 128
tree_depth = 12
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.222156524658203 | KNN Loss: 6.2308759689331055 | BCE Loss: 1.9912800788879395
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.250527381896973 | KNN Loss: 6.230742454528809 | BCE Loss: 2.019784927368164
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.177160263061523 | KNN Loss: 6.230837821960449 | BCE Loss: 1.9463227987289429
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.158607482910156 | KNN Loss: 6.2306413650512695 | BCE Loss: 1.9279663562774658
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.189087867736816 | KNN Loss: 6.230562686920166 | BCE Loss: 1.9585249423980713
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.206348419189453 | KNN Loss: 6.230489253997803 | BCE Loss: 1.97585928440094
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.168549537658691 | KNN Loss: 6.230339050292969 | BCE Loss: 1.9382104873657227
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.182073593139648 | KNN Loss: 6.230133056640625 | BCE Loss: 1.951940

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 7.244460582733154 | KNN Loss: 6.120423793792725 | BCE Loss: 1.1240366697311401
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 7.239505767822266 | KNN Loss: 6.110269069671631 | BCE Loss: 1.1292364597320557
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 7.203968048095703 | KNN Loss: 6.089426040649414 | BCE Loss: 1.11454176902771
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 7.219274044036865 | KNN Loss: 6.091695785522461 | BCE Loss: 1.1275783777236938
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 7.180758476257324 | KNN Loss: 6.0650410652160645 | BCE Loss: 1.1157176494598389
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 7.187288284301758 | KNN Loss: 6.0549516677856445 | BCE Loss: 1.1323363780975342
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 7.148846626281738 | KNN Loss: 6.03350305557251 | BCE Loss: 1.115343689918518
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 7.079246997833252 | KNN Loss: 5.996521472930908 | BCE Loss: 

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 6.100081443786621 | KNN Loss: 5.0664381980896 | BCE Loss: 1.033643126487732
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 6.133056640625 | KNN Loss: 5.060835361480713 | BCE Loss: 1.0722215175628662
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 6.144747734069824 | KNN Loss: 5.062102794647217 | BCE Loss: 1.0826447010040283
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 6.1050519943237305 | KNN Loss: 5.070614814758301 | BCE Loss: 1.0344369411468506
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 6.1240739822387695 | KNN Loss: 5.068467617034912 | BCE Loss: 1.0556062459945679
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 6.120849609375 | KNN Loss: 5.079685688018799 | BCE Loss: 1.0411639213562012
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 6.108098983764648 | KNN Loss: 5.069944858551025 | BCE Loss: 1.038154125213623
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 6.146769046783447 | KNN Loss: 5.057798862457275 | BCE Loss: 1.088

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 6.132634162902832 | KNN Loss: 5.058141708374023 | BCE Loss: 1.0744925737380981
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 6.13221549987793 | KNN Loss: 5.069360256195068 | BCE Loss: 1.0628553628921509
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 6.0787672996521 | KNN Loss: 5.034670352935791 | BCE Loss: 1.0440969467163086
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 6.0907511711120605 | KNN Loss: 5.040624618530273 | BCE Loss: 1.050126552581787
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 6.109333515167236 | KNN Loss: 5.041493892669678 | BCE Loss: 1.067839503288269
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 6.110171318054199 | KNN Loss: 5.057861804962158 | BCE Loss: 1.052309274673462
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 6.108434677124023 | KNN Loss: 5.054862976074219 | BCE Loss: 1.0535714626312256
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 6.096025466918945 | KNN Loss: 5.044345855712891 | BCE Loss: 1.

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 6.101199626922607 | KNN Loss: 5.0482258796691895 | BCE Loss: 1.0529736280441284
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 6.084390163421631 | KNN Loss: 5.035100936889648 | BCE Loss: 1.0492892265319824
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 6.100293159484863 | KNN Loss: 5.055597305297852 | BCE Loss: 1.0446959733963013
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 6.0953521728515625 | KNN Loss: 5.036495208740234 | BCE Loss: 1.0588572025299072
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 6.095643997192383 | KNN Loss: 5.040556907653809 | BCE Loss: 1.0550869703292847
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 6.09891414642334 | KNN Loss: 5.032977104187012 | BCE Loss: 1.0659370422363281
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 6.089578628540039 | KNN Loss: 5.052615642547607 | BCE Loss: 1.036962866783142
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 6.122159004211426 | KNN Loss: 5.065701961517334 | BCE Los

...

Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 6.052653789520264 | KNN Loss: 5.023716449737549 | BCE Loss: 1.0289373397827148
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 6.122894287109375 | KNN Loss: 5.069304943084717 | BCE Loss: 1.0535892248153687
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 6.049287796020508 | KNN Loss: 5.017678737640381 | BCE Loss: 1.031609296798706
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 6.062922477722168 | KNN Loss: 5.031908988952637 | BCE Loss: 1.0310134887695312
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 6.0268168449401855 | KNN Loss: 5.01405668258667 | BCE Loss: 1.0127602815628052
Epoch 450 / 500 | iteration 5 / 30 | Total Loss: 6.042359352111816 | KNN Loss: 5.026082992553711 | BCE Loss: 1.0162763595581055
Epoch 450 / 500 | iteration 10 / 30 | Total Loss: 6.0299601554870605 | KNN Loss: 5.010076522827148 | BCE Loss: 1.0198837518692017
Epoch 450 / 500 | iteration 15 / 30 | Total Loss: 6.058023929595947 | KNN Loss: 5.025526523590088 |

Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 6.033576965332031 | KNN Loss: 5.009700775146484 | BCE Loss: 1.0238761901855469
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 6.110397815704346 | KNN Loss: 5.0677337646484375 | BCE Loss: 1.0426640510559082
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 6.038633346557617 | KNN Loss: 5.005651950836182 | BCE Loss: 1.0329811573028564
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 6.037411689758301 | KNN Loss: 5.013095855712891 | BCE Loss: 1.024315595626831
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 6.04649019241333 | KNN Loss: 5.004507541656494 | BCE Loss: 1.041982650756836
Epoch 460 / 500 | iteration 25 / 30 | Total Loss: 6.074123859405518 | KNN Loss: 5.028656005859375 | BCE Loss: 1.045467734336853
Epoch 461 / 500 | iteration 0 / 30 | Total Loss: 6.081554889678955 | KNN Loss: 5.041596412658691 | BCE Loss: 1.0399584770202637
Epoch 461 / 500 | iteration 5 / 30 | Total Loss: 6.045753479003906 | KNN Loss: 5.027709007263184 | BCE 

Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 6.034987926483154 | KNN Loss: 5.00231409072876 | BCE Loss: 1.0326738357543945
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 6.0556440353393555 | KNN Loss: 5.0288166999816895 | BCE Loss: 1.0268275737762451
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 6.021641731262207 | KNN Loss: 5.002894401550293 | BCE Loss: 1.018747329711914
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 6.066617012023926 | KNN Loss: 5.026043891906738 | BCE Loss: 1.040573239326477
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 6.042706489562988 | KNN Loss: 5.02046012878418 | BCE Loss: 1.0222465991973877
Epoch 471 / 500 | iteration 15 / 30 | Total Loss: 6.03903865814209 | KNN Loss: 5.010788917541504 | BCE Loss: 1.0282495021820068
Epoch 471 / 500 | iteration 20 / 30 | Total Loss: 6.0509467124938965 | KNN Loss: 5.029558181762695 | BCE Loss: 1.0213885307312012
Epoch 471 / 500 | iteration 25 / 30 | Total Loss: 6.046949863433838 | KNN Loss: 5.011531352996826 | B

Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 6.034120082855225 | KNN Loss: 5.0099592208862305 | BCE Loss: 1.0241607427597046
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 6.052731513977051 | KNN Loss: 5.0125813484191895 | BCE Loss: 1.0401499271392822
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 6.090795040130615 | KNN Loss: 5.0254292488098145 | BCE Loss: 1.0653659105300903
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 6.014037132263184 | KNN Loss: 5.001946926116943 | BCE Loss: 1.0120902061462402
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 6.0226545333862305 | KNN Loss: 5.007966995239258 | BCE Loss: 1.0146877765655518
Epoch 482 / 500 | iteration 5 / 30 | Total Loss: 6.066821098327637 | KNN Loss: 5.047447204589844 | BCE Loss: 1.019373893737793
Epoch 482 / 500 | iteration 10 / 30 | Total Loss: 6.072332859039307 | KNN Loss: 5.006659507751465 | BCE Loss: 1.0656733512878418
Epoch 482 / 500 | iteration 15 / 30 | Total Loss: 6.0557942390441895 | KNN Loss: 5.0212788581848

Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 6.100092887878418 | KNN Loss: 5.060546875 | BCE Loss: 1.039546012878418
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 6.032808303833008 | KNN Loss: 5.01011848449707 | BCE Loss: 1.022689938545227
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 6.03571891784668 | KNN Loss: 5.011983871459961 | BCE Loss: 1.0237348079681396
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 6.067623138427734 | KNN Loss: 5.043232440948486 | BCE Loss: 1.024390697479248
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 6.050644874572754 | KNN Loss: 5.029106616973877 | BCE Loss: 1.0215381383895874
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 5.991954803466797 | KNN Loss: 5.000690460205078 | BCE Loss: 0.9912642240524292
Epoch 493 / 500 | iteration 0 / 30 | Total Loss: 6.029138565063477 | KNN Loss: 5.0149712562561035 | BCE Loss: 1.014167070388794
Epoch 493 / 500 | iteration 5 / 30 | Total Loss: 6.039982795715332 | KNN Loss: 4.9985671043396 | BCE Loss: 1.04

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.0212,  4.1637,  2.2793,  2.4290,  2.3282,  0.7648,  2.8325,  1.9331,
          1.9076,  2.1568,  2.3861,  2.4074,  0.6492,  1.8556,  1.3833,  1.1246,
          2.4802,  3.3786,  2.0662,  2.5517,  1.3852,  3.1902,  2.0022,  2.8558,
          2.2824,  1.8604,  2.3070,  1.5351,  1.6248,  0.4055, -0.2656,  0.5114,
          0.3085,  1.0195,  1.6012,  1.5376,  1.1399,  3.5268,  0.9174,  1.3869,
          1.0135, -0.7747, -0.2560,  2.5865,  2.2172,  0.8223, -0.1455,  0.1383,
          1.5418,  2.3296,  1.9156,  0.1940,  1.0721,  0.4884, -0.6085,  1.2113,
          1.6052,  1.4865,  0.9008,  1.9952,  0.6163,  0.9264,  0.1962,  1.7886,
          1.4019,  1.8067, -1.8601,  0.4402,  2.3858,  1.7615,  2.2435,  0.4592,
          1.4922,  2.6230,  2.1548,  1.4235,  0.3650,  0.8210,  0.3345,  1.1884,
          0.1844,  0.4736,  1.4085, -0.2858, -0.0079, -1.0756, -2.3881, -0.1343,
          0.5750, -1.8682,  0.5242, -0.0994, -0.5039, -0.8989,  0.6663,  1.3529,
         -0.6817, -0.5918,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 79.29it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
layer 9: 0.0
layer 10: 0.0
Epoch: 00 | Batch: 000 / 029 | Total loss: 9.631 | Reg loss: 0.014 | Tree loss: 9.631 | Accuracy: 0.000000 | 6.98 sec/iter
Epoch: 00 | Batch: 001 / 029 | Total loss: 9.625 | Reg loss: 0.013 | Tree loss: 9.625 | Accuracy: 0.000000 | 6.279 sec/iter
Epoch: 00 | Batch: 002 / 029 | Total loss: 9.619 | Reg loss: 0.012 | Tree loss: 9.619 | Accuracy: 0.000000 | 6.027 sec/iter
Epoch: 00 | Batch: 003 / 029 | Total loss: 9.614 | Reg loss: 0.011 | Tree loss: 9.614 | Accuracy: 0.000000 | 5.884 sec/iter
Epoch: 00 | Batch: 004 / 029 | Total loss: 9.608 | Reg loss: 0.010 | Tree loss: 9.608 | Accuracy: 0.000000 | 6.131 sec/iter
Epoch: 00 | Batch: 005 / 029 | Total loss: 9.602 | Reg loss: 0.009 | Tree loss: 9.602 | Accuracy: 0.000000 | 6.273 sec/iter
Epoch: 00 | Batch: 006 / 029 | Total loss: 9.596 | Reg loss: 0.009 | Tree loss: 9.596 | A

Epoch: 02 | Batch: 002 / 029 | Total loss: 9.463 | Reg loss: 0.009 | Tree loss: 9.463 | Accuracy: 0.529297 | 6.701 sec/iter
Epoch: 02 | Batch: 003 / 029 | Total loss: 9.463 | Reg loss: 0.009 | Tree loss: 9.463 | Accuracy: 0.492188 | 6.708 sec/iter
Epoch: 02 | Batch: 004 / 029 | Total loss: 9.457 | Reg loss: 0.009 | Tree loss: 9.457 | Accuracy: 0.474609 | 6.715 sec/iter
Epoch: 02 | Batch: 005 / 029 | Total loss: 9.450 | Reg loss: 0.009 | Tree loss: 9.450 | Accuracy: 0.505859 | 6.721 sec/iter
Epoch: 02 | Batch: 006 / 029 | Total loss: 9.443 | Reg loss: 0.010 | Tree loss: 9.443 | Accuracy: 0.498047 | 6.728 sec/iter
Epoch: 02 | Batch: 007 / 029 | Total loss: 9.443 | Reg loss: 0.010 | Tree loss: 9.443 | Accuracy: 0.458984 | 6.733 sec/iter
Epoch: 02 | Batch: 008 / 029 | Total loss: 9.431 | Reg loss: 0.010 | Tree loss: 9.431 | Accuracy: 0.517578 | 6.74 sec/iter
Epoch: 02 | Batch: 009 / 029 | Total loss: 9.427 | Reg loss: 0.011 | Tree loss: 9.427 | Accuracy: 0.484375 | 6.746 sec/iter
Epoch: 02

Epoch: 04 | Batch: 005 / 029 | Total loss: 9.220 | Reg loss: 0.015 | Tree loss: 9.220 | Accuracy: 0.505859 | 6.687 sec/iter
Epoch: 04 | Batch: 006 / 029 | Total loss: 9.204 | Reg loss: 0.015 | Tree loss: 9.204 | Accuracy: 0.521484 | 6.69 sec/iter
Epoch: 04 | Batch: 007 / 029 | Total loss: 9.183 | Reg loss: 0.015 | Tree loss: 9.183 | Accuracy: 0.527344 | 6.693 sec/iter
Epoch: 04 | Batch: 008 / 029 | Total loss: 9.179 | Reg loss: 0.016 | Tree loss: 9.179 | Accuracy: 0.476562 | 6.695 sec/iter
Epoch: 04 | Batch: 009 / 029 | Total loss: 9.157 | Reg loss: 0.016 | Tree loss: 9.157 | Accuracy: 0.498047 | 6.695 sec/iter
Epoch: 04 | Batch: 010 / 029 | Total loss: 9.139 | Reg loss: 0.017 | Tree loss: 9.139 | Accuracy: 0.515625 | 6.696 sec/iter
Epoch: 04 | Batch: 011 / 029 | Total loss: 9.125 | Reg loss: 0.017 | Tree loss: 9.125 | Accuracy: 0.482422 | 6.697 sec/iter
Epoch: 04 | Batch: 012 / 029 | Total loss: 9.099 | Reg loss: 0.017 | Tree loss: 9.099 | Accuracy: 0.515625 | 6.697 sec/iter
Epoch: 04

Epoch: 06 | Batch: 008 / 029 | Total loss: 8.658 | Reg loss: 0.020 | Tree loss: 8.658 | Accuracy: 0.505859 | 6.978 sec/iter
Epoch: 06 | Batch: 009 / 029 | Total loss: 8.648 | Reg loss: 0.020 | Tree loss: 8.648 | Accuracy: 0.494141 | 6.979 sec/iter
Epoch: 06 | Batch: 010 / 029 | Total loss: 8.626 | Reg loss: 0.021 | Tree loss: 8.626 | Accuracy: 0.474609 | 6.979 sec/iter
Epoch: 06 | Batch: 011 / 029 | Total loss: 8.596 | Reg loss: 0.021 | Tree loss: 8.596 | Accuracy: 0.503906 | 6.98 sec/iter
Epoch: 06 | Batch: 012 / 029 | Total loss: 8.548 | Reg loss: 0.021 | Tree loss: 8.548 | Accuracy: 0.535156 | 6.979 sec/iter
Epoch: 06 | Batch: 013 / 029 | Total loss: 8.515 | Reg loss: 0.022 | Tree loss: 8.515 | Accuracy: 0.509766 | 6.978 sec/iter
Epoch: 06 | Batch: 014 / 029 | Total loss: 8.493 | Reg loss: 0.022 | Tree loss: 8.493 | Accuracy: 0.472656 | 6.978 sec/iter
Epoch: 06 | Batch: 015 / 029 | Total loss: 8.478 | Reg loss: 0.022 | Tree loss: 8.478 | Accuracy: 0.466797 | 6.976 sec/iter
Epoch: 06

Epoch: 08 | Batch: 011 / 029 | Total loss: 7.915 | Reg loss: 0.024 | Tree loss: 7.915 | Accuracy: 0.509766 | 7.1 sec/iter
Epoch: 08 | Batch: 012 / 029 | Total loss: 7.900 | Reg loss: 0.025 | Tree loss: 7.900 | Accuracy: 0.503906 | 7.092 sec/iter
Epoch: 08 | Batch: 013 / 029 | Total loss: 7.852 | Reg loss: 0.025 | Tree loss: 7.852 | Accuracy: 0.484375 | 7.084 sec/iter
Epoch: 08 | Batch: 014 / 029 | Total loss: 7.857 | Reg loss: 0.025 | Tree loss: 7.857 | Accuracy: 0.474609 | 7.084 sec/iter
Epoch: 08 | Batch: 015 / 029 | Total loss: 7.794 | Reg loss: 0.026 | Tree loss: 7.794 | Accuracy: 0.496094 | 7.083 sec/iter
Epoch: 08 | Batch: 016 / 029 | Total loss: 7.736 | Reg loss: 0.026 | Tree loss: 7.736 | Accuracy: 0.513672 | 7.081 sec/iter
Epoch: 08 | Batch: 017 / 029 | Total loss: 7.737 | Reg loss: 0.026 | Tree loss: 7.737 | Accuracy: 0.474609 | 7.08 sec/iter
Epoch: 08 | Batch: 018 / 029 | Total loss: 7.729 | Reg loss: 0.027 | Tree loss: 7.729 | Accuracy: 0.480469 | 7.078 sec/iter
Epoch: 08 |

...

Epoch: 90 | Batch: 003 / 029 | Total loss: 1.292 | Reg loss: 0.029 | Tree loss: 1.292 | Accuracy: 0.529297 | 7.103 sec/iter
Epoch: 90 | Batch: 004 / 029 | Total loss: 1.313 | Reg loss: 0.029 | Tree loss: 1.313 | Accuracy: 0.482422 | 7.103 sec/iter
Epoch: 90 | Batch: 005 / 029 | Total loss: 1.292 | Reg loss: 0.029 | Tree loss: 1.292 | Accuracy: 0.482422 | 7.104 sec/iter
Epoch: 90 | Batch: 006 / 029 | Total loss: 1.252 | Reg loss: 0.029 | Tree loss: 1.252 | Accuracy: 0.525391 | 7.104 sec/iter
Epoch: 90 | Batch: 007 / 029 | Total loss: 1.239 | Reg loss: 0.029 | Tree loss: 1.239 | Accuracy: 0.537109 | 7.104 sec/iter
Epoch: 90 | Batch: 008 / 029 | Total loss: 1.269 | Reg loss: 0.029 | Tree loss: 1.269 | Accuracy: 0.498047 | 7.103 sec/iter
Epoch: 90 | Batch: 009 / 029 | Total loss: 1.291 | Reg loss: 0.029 | Tree loss: 1.291 | Accuracy: 0.476562 | 7.103 sec/iter
Epoch: 90 | Batch: 010 / 029 | Total loss: 1.279 | Reg loss: 0.029 | Tree loss: 1.279 | Accuracy: 0.470703 | 7.103 sec/iter
Epoch: 9

Epoch: 92 | Batch: 006 / 029 | Total loss: 1.302 | Reg loss: 0.028 | Tree loss: 1.302 | Accuracy: 0.480469 | 7.099 sec/iter
Epoch: 92 | Batch: 007 / 029 | Total loss: 1.271 | Reg loss: 0.028 | Tree loss: 1.271 | Accuracy: 0.496094 | 7.099 sec/iter
Epoch: 92 | Batch: 008 / 029 | Total loss: 1.293 | Reg loss: 0.029 | Tree loss: 1.293 | Accuracy: 0.468750 | 7.099 sec/iter
Epoch: 92 | Batch: 009 / 029 | Total loss: 1.271 | Reg loss: 0.029 | Tree loss: 1.271 | Accuracy: 0.490234 | 7.099 sec/iter
Epoch: 92 | Batch: 010 / 029 | Total loss: 1.250 | Reg loss: 0.029 | Tree loss: 1.250 | Accuracy: 0.501953 | 7.099 sec/iter
Epoch: 92 | Batch: 011 / 029 | Total loss: 1.232 | Reg loss: 0.029 | Tree loss: 1.232 | Accuracy: 0.513672 | 7.099 sec/iter
Epoch: 92 | Batch: 012 / 029 | Total loss: 1.221 | Reg loss: 0.029 | Tree loss: 1.221 | Accuracy: 0.515625 | 7.099 sec/iter
Epoch: 92 | Batch: 013 / 029 | Total loss: 1.250 | Reg loss: 0.029 | Tree loss: 1.250 | Accuracy: 0.470703 | 7.099 sec/iter
Epoch: 9

Epoch: 94 | Batch: 009 / 029 | Total loss: 1.249 | Reg loss: 0.028 | Tree loss: 1.249 | Accuracy: 0.509766 | 7.092 sec/iter
Epoch: 94 | Batch: 010 / 029 | Total loss: 1.262 | Reg loss: 0.028 | Tree loss: 1.262 | Accuracy: 0.474609 | 7.092 sec/iter
Epoch: 94 | Batch: 011 / 029 | Total loss: 1.238 | Reg loss: 0.028 | Tree loss: 1.238 | Accuracy: 0.498047 | 7.092 sec/iter
Epoch: 94 | Batch: 012 / 029 | Total loss: 1.255 | Reg loss: 0.028 | Tree loss: 1.255 | Accuracy: 0.484375 | 7.092 sec/iter
Epoch: 94 | Batch: 013 / 029 | Total loss: 1.245 | Reg loss: 0.029 | Tree loss: 1.245 | Accuracy: 0.476562 | 7.092 sec/iter
Epoch: 94 | Batch: 014 / 029 | Total loss: 1.226 | Reg loss: 0.029 | Tree loss: 1.226 | Accuracy: 0.511719 | 7.092 sec/iter
Epoch: 94 | Batch: 015 / 029 | Total loss: 1.259 | Reg loss: 0.029 | Tree loss: 1.259 | Accuracy: 0.451172 | 7.092 sec/iter
Epoch: 94 | Batch: 016 / 029 | Total loss: 1.195 | Reg loss: 0.029 | Tree loss: 1.195 | Accuracy: 0.525391 | 7.092 sec/iter
Epoch: 9

Epoch: 96 | Batch: 012 / 029 | Total loss: 1.220 | Reg loss: 0.028 | Tree loss: 1.220 | Accuracy: 0.519531 | 7.08 sec/iter
Epoch: 96 | Batch: 013 / 029 | Total loss: 1.212 | Reg loss: 0.028 | Tree loss: 1.212 | Accuracy: 0.498047 | 7.079 sec/iter
Epoch: 96 | Batch: 014 / 029 | Total loss: 1.234 | Reg loss: 0.028 | Tree loss: 1.234 | Accuracy: 0.484375 | 7.079 sec/iter
Epoch: 96 | Batch: 015 / 029 | Total loss: 1.212 | Reg loss: 0.028 | Tree loss: 1.212 | Accuracy: 0.509766 | 7.078 sec/iter
Epoch: 96 | Batch: 016 / 029 | Total loss: 1.249 | Reg loss: 0.028 | Tree loss: 1.249 | Accuracy: 0.457031 | 7.077 sec/iter
Epoch: 96 | Batch: 017 / 029 | Total loss: 1.196 | Reg loss: 0.029 | Tree loss: 1.196 | Accuracy: 0.503906 | 7.076 sec/iter
Epoch: 96 | Batch: 018 / 029 | Total loss: 1.210 | Reg loss: 0.029 | Tree loss: 1.210 | Accuracy: 0.501953 | 7.075 sec/iter
Epoch: 96 | Batch: 019 / 029 | Total loss: 1.205 | Reg loss: 0.029 | Tree loss: 1.205 | Accuracy: 0.490234 | 7.074 sec/iter
Epoch: 96

Epoch: 98 | Batch: 015 / 029 | Total loss: 1.184 | Reg loss: 0.028 | Tree loss: 1.184 | Accuracy: 0.533203 | 7.035 sec/iter
Epoch: 98 | Batch: 016 / 029 | Total loss: 1.194 | Reg loss: 0.028 | Tree loss: 1.194 | Accuracy: 0.519531 | 7.034 sec/iter
Epoch: 98 | Batch: 017 / 029 | Total loss: 1.207 | Reg loss: 0.028 | Tree loss: 1.207 | Accuracy: 0.486328 | 7.033 sec/iter
Epoch: 98 | Batch: 018 / 029 | Total loss: 1.233 | Reg loss: 0.028 | Tree loss: 1.233 | Accuracy: 0.468750 | 7.032 sec/iter
Epoch: 98 | Batch: 019 / 029 | Total loss: 1.229 | Reg loss: 0.028 | Tree loss: 1.229 | Accuracy: 0.462891 | 7.031 sec/iter
Epoch: 98 | Batch: 020 / 029 | Total loss: 1.203 | Reg loss: 0.028 | Tree loss: 1.203 | Accuracy: 0.490234 | 7.031 sec/iter
Epoch: 98 | Batch: 021 / 029 | Total loss: 1.180 | Reg loss: 0.028 | Tree loss: 1.180 | Accuracy: 0.503906 | 7.03 sec/iter
Epoch: 98 | Batch: 022 / 029 | Total loss: 1.208 | Reg loss: 0.029 | Tree loss: 1.208 | Accuracy: 0.458984 | 7.029 sec/iter
Epoch: 98

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 11.886291486291487


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 3465


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

============== Pattern 646 ==============
============== Pattern 647 ==============
============== Pattern 648 ==============
============== Pattern 649 ==============
============== Pattern 650 ==============
============== Pattern 651 ==============
============== Pattern 652 ==============
============== Pattern 653 ==============
============== Pattern 654 ==============
============== Pattern 655 ==============
============== Pattern 656 ==============
============== Pattern 657 ==============
============== Pattern 658 ==============
============== Pattern 659 ==============
============== Pattern 660 ==============
============== Pattern 661 ==============
============== Pattern 662 ==============
============== Pattern 663 ==============
============== Pattern 664 ==============
============== Pattern 665 ==============
============== Pattern 666 ==============
============== Pattern 667 ==============
============== Pattern 668 ==============
============== Pattern 669 =======

============== Pattern 1276 ==============
============== Pattern 1277 ==============
============== Pattern 1278 ==============
============== Pattern 1279 ==============
============== Pattern 1280 ==============
============== Pattern 1281 ==============
============== Pattern 1282 ==============
============== Pattern 1283 ==============
============== Pattern 1284 ==============
============== Pattern 1285 ==============
============== Pattern 1286 ==============
============== Pattern 1287 ==============
============== Pattern 1288 ==============
============== Pattern 1289 ==============
============== Pattern 1290 ==============
============== Pattern 1291 ==============
============== Pattern 1292 ==============
============== Pattern 1293 ==============
============== Pattern 1294 ==============
============== Pattern 1295 ==============
============== Pattern 1296 ==============
============== Pattern 1297 ==============
============== Pattern 1298 ==============
===========

============== Pattern 1802 ==============
============== Pattern 1803 ==============
============== Pattern 1804 ==============
============== Pattern 1805 ==============
============== Pattern 1806 ==============
============== Pattern 1807 ==============
============== Pattern 1808 ==============
============== Pattern 1809 ==============
============== Pattern 1810 ==============
============== Pattern 1811 ==============
============== Pattern 1812 ==============
============== Pattern 1813 ==============
============== Pattern 1814 ==============
============== Pattern 1815 ==============
============== Pattern 1816 ==============
============== Pattern 1817 ==============
============== Pattern 1818 ==============
============== Pattern 1819 ==============
============== Pattern 1820 ==============
============== Pattern 1821 ==============
============== Pattern 1822 ==============
============== Pattern 1823 ==============
============== Pattern 1824 ==============
===========

============== Pattern 2801 ==============
============== Pattern 2802 ==============
============== Pattern 2803 ==============
============== Pattern 2804 ==============
============== Pattern 2805 ==============
============== Pattern 2806 ==============
============== Pattern 2807 ==============
============== Pattern 2808 ==============
============== Pattern 2809 ==============
============== Pattern 2810 ==============
============== Pattern 2811 ==============
============== Pattern 2812 ==============
============== Pattern 2813 ==============
============== Pattern 2814 ==============
============== Pattern 2815 ==============
============== Pattern 2816 ==============
============== Pattern 2817 ==============
============== Pattern 2818 ==============
============== Pattern 2819 ==============
============== Pattern 2820 ==============
============== Pattern 2821 ==============
============== Pattern 2822 ==============
============== Pattern 2823 ==============
===========

var comprehensibility: 17.433388429752068
minimum comprehensibility: 30
maximum comprehensibility: 68
